# EDA

## Загрузка данных

In [2]:
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_csv('cian_rent.csv', low_memory=False)
df.head(2)

,author,author_type,agent_name,url,location,deal_type,accommodation_type,price,year_of_construction,house_material_type,...,sewage_system,bathroom,living_meters,floors_count,phone,district,underground,street,house_number,creation_date
0,NaN,realtor_based,NaN,https://www.cian.ru/rent/flat/319524074/,Москва,rent_long,flat,290300.0,NaN,monolithBrick,...,NaN,NaN,60.1,28,NaN,NaN,"Багратионовская, Парк Победы, Филёвский парк",NaN,NaN,2025-07-08
1,NaN,realtor_based,NaN,https://www.cian.ru/rent/flat/313850443/,Москва,rent_long,flat,200600.0,2005.0,panel,...,NaN,NaN,53.3,22,NaN,NaN,"Поклонная, Ломоносовский проспект, Киевская",NaN,NaN,2025-02-18


## Обработка пропущенных значений

In [3]:
# Удаляем полностью пустые колонки
df.dropna(axis=1, how='all', inplace=True)

# Заполняем пропуски в числовых данных медианой
numeric_cols = ['price', 'living_meters', 'floors_count']
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

# Категориальные колонки: заполняем "unknown" 
categorical_cols = ['house_material_type', 'heating_type', 'district']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].fillna('unknown')
      

# Удаляем строки, где нет цены или площади
df = df.dropna(subset=['price', 'living_meters'], how='any')

df.head(3)

,author_type,url,location,deal_type,accommodation_type,price,year_of_construction,house_material_type,living_meters,floors_count,underground,creation_date
0,realtor_based,https://www.cian.ru/rent/flat/319524074/,Москва,rent_long,flat,290300.0,NaN,monolithBrick,60.1,28,"Багратионовская, Парк Победы, Филёвский парк",2025-07-08
1,realtor_based,https://www.cian.ru/rent/flat/313850443/,Москва,rent_long,flat,200600.0,2005.0,panel,53.3,22,"Поклонная, Ломоносовский проспект, Киевская",2025-02-18
2,realtor_based,https://www.cian.ru/rent/flat/317523322/,Москва,rent_long,flat,84300.0,1960.0,panel,25.0,5,"Ломоносовский проспект, Минская, Минская",2025-05-14


## Приведение типов данных

In [ ]:
# Преобразуем цену в число
if 'price' in df.columns:
    df['price'] = df['price'].astype(str).str.replace(' ', '').astype(float)

# Дата создания объявления в datetime
if 'creation_date' in df.columns:
    df['creation_date'] = pd.to_datetime(df['creation_date'], errors='coerce')

if 'floors_count' in df.columns:
    df['floors_count'] = df['floors_count'].astype(int)

In [21]:
# Удаляем лишние пробелы и кавычки в адресах
text_cols = ['district', 'street', 'house_number']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].str.strip(' "\'')

# Приводим названия районов к одному регистру (например, "филёвский парк" -> "Филёвский парк")
if 'district' in df.columns:
    df['district'] = df['district'].str.title()

## Обработка аномалий

In [ ]:
# Удаляем выбросы в цене (например, квартиры дешевле 20 тыс. или дороже 500 тыс.)
if 'price' in df.columns:
    df = df[(df['price'] > 20_000) & (df['price'] < 500_000)]

# Проверяем метраж на аномалии (например, квартиры < 10 м² или > 5000 м²)
if 'living_meters' in df.columns:
    df = df[(df['living_meters'] > 10) & (df['living_meters'] < 500)]

# Фиксим этажность: если этаж > floors_count, заменяем на floors_count
if 'floor' in df.columns and 'floors_count' in df.columns:
    df['floor'] = np.where(df['floor'] > df['floors_count'], df['floors_count'], df['floor'])

## Разделение сложных полей

In [28]:
if 'floor' in df.columns and 'floors_count' in df.columns:
    df['floor'] = df['floor'].fillna(1)
    
    # Заменяем NaN в 'floors_count' на медианное значение
    df['floors_count'] = df['floors_count'].fillna(df['floors_count'].median())
    
    # Преобразуем оба столбца в int
    df['floor'] = df['floor'].astype(int)
    df['floors_count'] = df['floors_count'].astype(int)
    
    # Исправляем этаж, если он превышает общее число этажей
    df['floor'] = np.where(
        df['floor'] > df['floors_count'],
        df['floors_count'],
        df['floor']
    )

## Сохранение очищенных данных

In [29]:
# Удаляем промежуточные колонки
df = df.drop(columns=['underground_list'], errors='ignore')

# Сохраняем в CSV
df.to_csv('cleaned_data.csv', index=False)
print("Данные очищены и сохранены!")

Данные очищены и сохранены!
